# Principais métodos de chains com LCEL

A interface padrão de LCEL inclui os seguintes métodos:

- **stream:** transmitir de volta fragmentos da resposta
- **invoke:** chamar a cadeia com um input
- **batch:** chamar a cadeia com uma lista de inputs

Esses também possuem métodos assíncronos correspondentes que devem ser usados com a sintaxe `asyncio await` para concorrência:

- **astream:** transmitir de volta fragmentos da resposta de forma assíncrona
- **ainvoke:** chamar a cadeia com um input de forma assíncrona
- **abatch:** chamar a cadeia com uma lista de inputs de forma assíncrona
- **astream_log:** transmitir de volta etapas intermediárias à medida que acontecem, além da resposta final
- **astream_events:** transmitir eventos beta à medida que acontecem na cadeia

In [2]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre o assunto: {assunto}")

chain = prompt | model

## Invoke

O invoke é o método básico para inserir uma input na cadeia e receber uma resposta

In [4]:
chain.invoke({'assunto': 'cachorrinhos'})

AIMessage(content='Cachorrinhos são seres cheios de amor e lealdade, capazes de alegrar nossos dias com sua presença e companhia incondicional.', response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 20, 'total_tokens': 59, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-743f0750-7bab-44de-85c4-aa67db1adbba-0')

In [5]:
chain.invoke('cachorrinhos')

AIMessage(content='Cachorrinhos são como pequenos raios de sol que iluminam nossos dias com amor e alegria.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 20, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d41c477f-9615-4d2b-9f62-62c6a640fbc7-0')

## Stream

Para recebermos uma saída conforme ela é gerada pelo modelo utilizamos o stream


In [6]:
for chunck in chain.stream({'assunto': 'cachorrinho'}):
    print(chunck.content, end='', flush=True)

O cachorrinho é o melhor amigo do homem, sempre fiel e amoroso em todas as horas.

## Batch

Para fazermos múltimpas requisições em paralelo utilizamos o batch

In [7]:
chain.batch([{'assunto': 'cachorrinhos'}, {'assunto': 'gatinho'}, {'assunto': 'cavalinhos'}])

[AIMessage(content='Cachorrinhos são companheiros leais que enchem nossos dias de amor e alegria com suas travessuras e carinho.', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 20, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d0f46091-5b66-48af-aa73-0f0ac74a83c3-0'),
 AIMessage(content='"O ronronar de um gatinho é a canção mais doce que acalma a alma."', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 19, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_n

In [8]:
chain.batch([{'assunto': 'cachorrinhos'}, {'assunto': 'gatinho'}, {'assunto': 'cavalinhos'}], config={'max_concurrency': 5})

[AIMessage(content='Os cachorrinhos são amigos leais que aquecem nossos corações com sua alegria e amor incondicional.', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 20, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-326a2323-9cc5-428b-a8cb-613274c87221-0'),
 AIMessage(content='O gatinho é o companheiro perfeito para alegrar nossos dias com sua fofura e carinho.', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 19, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},

## Ainvoke

Todos os métodos possuem assíncronos correspondentes que devem ser usados com a sintaxe `asyncio await` para concorrência

In [9]:
import asyncio

async def processa_chain(input):
    resposta = await chain.ainvoke(input)
    return resposta

In [10]:
task1 = asyncio.create_task(processa_chain({'assunto': 'gatinho'}))
task2 = asyncio.create_task(processa_chain({'assunto': 'cavalinhos'}))
task3 = asyncio.create_task(processa_chain({'assunto': 'cachorrinhos'}))

await task1
await task2
await task3

AIMessage(content='Cachorrinhos são a prova de que o amor incondicional pode vir em formas peludas e com quatro patas.', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 20, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c4d1fee5-f08a-4fc3-8a02-60ea105055d5-0')

In [11]:
task2.result()

AIMessage(content='Os cavalinhos de brinquedo são a diversão preferida das crianças em parques e festas.', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 19, 'total_tokens': 44, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f92191f8-ad2f-47cf-90dd-0850bcd7b82e-0')

### Calculando a diferença de tempo ao utilizar async

In [12]:
from time import time

a = time()

task1 = asyncio.create_task(processa_chain({'assunto': 'gatinho'}))
task2 = asyncio.create_task(processa_chain({'assunto': 'cavalinhos'}))
task3 = asyncio.create_task(processa_chain({'assunto': 'cachorrinhos'}))

await task1
await task2
await task3

b = time()
print(b-a)

1.4262347221374512


In [13]:
from time import time

a = time()

task1 = chain.invoke({'assunto': 'gatinho'})
task2 = chain.invoke({'assunto': 'cavalinhos'})
task3 = chain.invoke({'assunto': 'cachorrinhos'})

b = time()
print(b-a)

2.151465892791748
